In [1]:
import pathlib
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

from c3.libraries import fidelities
from c3.parametermap import ParameterMap
from utils import *
from c3.experiment import Experiment
import c3.libraries.constants as constants
import c3.utils.qt_utils as qt_utils
import c3.libraries.algorithms as algorithms

2021-08-29 14:38:25.300828: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-29 14:38:25.300868: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
def findPeaks(x, y, N: int) -> Tuple:
    """
    Returns the x and y values of the N largest local maxima in y.
    """
    peaks = find_peaks(y)[0]
    peakX = x[peaks]
    peakY = y[peaks]
    maxIndices = peakY.argsort()[-N:][::-1]
    return peakX[maxIndices], peakY[maxIndices]


# plotting functions
def plotSignal(time, signal, filename=None, spectrum_cut=1e-4) -> None:
    """
    Plots a time dependent signal and its normalised frequency spectrum.

    Parameters
    ----------
    time
        timestamps
    signal
        signal value
    filename: str
        Optional name of the file to which the plot will be saved. If none,
        it will only be shown.
    spectrum_cut:
        If not None, only the part of the normalised spectrum will be plotted
        whose absolute square is larger than this value.

    Returns
    -------

    """
    # plot time domain
    time = time.flatten()
    signal = signal.flatten()
    print("time domain: ", time.shape, signal.shape)
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[0].set_title('Signal')
    axs[0].plot(time, signal)
    axs[0].set_xlabel('time')

    # calculate frequency spectrum
    freq_signal = np.fft.rfft(signal)
    if np.abs(np.max(freq_signal)) > 1e-14:
        normalised = freq_signal / np.max(freq_signal)
    else:
        normalised = freq_signal
    freq = np.fft.rfftfreq(len(time), time[-1] / len(time))
    print("frequency domain: ", freq.shape, freq_signal.shape)

    peakFrequencies, peakValues = findPeaks(freq, np.abs(normalised) ** 2, 4)
    print("peaks: ", np.sort(peakFrequencies))

    # cut spectrum if necessary
    if spectrum_cut is not None:
        limits = np.flatnonzero(np.abs(normalised) ** 2 > spectrum_cut)
        freq = freq[limits[0]:limits[-1]]
        normalised = normalised[limits[0]:limits[-1]]

    # plot frequency domain
    axs[1].set_title('Spectrum')
    axs[1].plot(freq, normalised.real, label="Re")
    axs[1].plot(freq, normalised.imag, label="Im")
    axs[1].plot(freq, np.abs(normalised) ** 2, label="Square")
    axs[1].set_xlabel('frequency')
    axs[1].legend()

    # show and save
    plt.tight_layout()
    if filename:
        print("saving plot in " + filename)
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()


def plotOccupations(
        experiment: Experiment,
        populations: np.array,
        gate_sequence: List[str],
        level_names: List[str] = None,
        filename: str = None,
) -> None:
    """
    Plots time dependent populations. They need to be calculated with `runTimeEvolution` first.

    Parameters
    ----------
    experiment: Experiment
        The experiment containing the model and propagators
    populations: np.array
        Population vector for each time step
    gate_sequence: List[str]
        List of gate names that will be applied to the state
    level_names: List[str]
        Optional list of names for the levels. If none, the default list
        from the experiment will be used.
    filename: str
        Optional name of the file to which the plot will be saved. If none,
        it will only be shown.

    Returns
    -------

    """
    # plot populations
    fig, axs = plt.subplots(1, 1)
    dt = experiment.ts[1] - experiment.ts[0]
    ts = np.linspace(0.0, dt * populations.shape[1], populations.shape[1])
    axs.plot(ts / 1e-9, populations.T)

    # plot vertical lines
    gate_steps = [experiment.partial_propagators[g].shape[0] for g in gate_sequence]
    for i in range(1, len(gate_steps)):
        gate_steps[i] += gate_steps[i - 1]
    gate_times = gate_steps * dt
    plt.vlines(gate_times / 1e-9, tf.reduce_min(populations), tf.reduce_max(populations),
               linestyles=':', colors="black")

    # set plot properties
    axs.tick_params(direction="in", left=True, right=True, top=False, bottom=True)
    axs.set_xlabel('Time [ns]')
    axs.set_ylabel('Population')
    plt.legend(level_names if level_names else experiment.pmap.model.state_labels)
    plt.tight_layout()

    # show and save
    if filename:
        print("saving plot in " + filename)
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()


def runExperiment(
        t_final: float,
        freq: float,
        anharm: float,
        carrier_freq: float,
        envelope: pulse.Envelope,
        optimisable_params: dict,
        algorithm: Callable,
        algorithm_options: {},
        file_suffix: str,
        directory: str = None,
        num_gates=1
) -> None:
    # preparation
    active_levels = 4
    occupied_levels = [0, 2]
    if directory is None:
        directory = "./output"
    output_dir = pathlib.Path(directory)
    output_dir.mkdir(parents=True, exist_ok=True)

    # model
    q1 = createQubit(1, 5, freq, -anharm)
    model = createModel([q1])
    generator = createGenerator(model)
    energies = q1.get_Hamiltonian().numpy().diagonal().real
    print("energies: ", [(energies[i + 1] - energies[i]) / (2 * np.pi) for i in range(len(energies) - 1)])

    # gate
    #envelope = createPWCGaussianPulse(t_final, t_final / 4, 30)
    ideal = qt_utils.np_kron_n([
        constants.Id,
        constants.X,
    ])
    gate = createSingleQubitGate("lower-X", t_final, carrier_freq, envelope, model, q1, ideal)
    gates = [gate] * num_gates
    gate_names = list(map(lambda g: g.get_key(), gates))

    # experiment
    exp = Experiment(pmap=ParameterMap(instructions=gates, model=model, generator=generator))
    exp.set_opt_gates(gate_names)
    unitaries = exp.compute_propagators()
    #print('unitaries: ', dict(map(lambda kv: (kv[0], kv[1].numpy().shape), unitaries.items())))

    # initial state
    init_state = createState(model, occupied_levels)
    #state = init_state.numpy().flatten()
    #print("initial state=", state, ", occupation=", exp.populations(state, model.lindbladian).numpy())

    # time evolution and signal before optimisation
    sequence = ["lower-X[0]"]
    signal = generator.generate_signals(gate)[getDrive(model, q1).name]
    plotSignal(signal['ts'].numpy(), signal['values'].numpy(), directory + f"/signal_before_{file_suffix}.png",
               spectrum_cut=1e-3)
    populations = runTimeEvolutionDefault(exp, init_state, sequence)
    plotOccupations(exp, populations, sequence, filename=directory + f"/populations_before_{file_suffix}.png")

    # optimise
    optimisable_gates = list(filter(lambda g: g.get_key() != "id[]", gates))
    gateset_opt_map = createOptimisableParameterMap(exp, optimisable_gates, optimisable_params)
    callback = lambda fidelity: print(fidelity)
    params_before, final_fidelity, params_after = optimise(
        exp, optimisable_gates,
        optimisable_parameters=gateset_opt_map,
        fidelity_fctn=fidelities.state_transfer_infid_set,
        fidelity_params={
            'psi_0': init_state[:active_levels],
            'active_levels': 4,
            'num_gates': len(gates)
        },
        #callback=callback,
        algorithm=algorithm,
        algo_options=algorithm_options,
        log_dir=(directory + ("/log_{0}_{1:.2f}/".format(file_suffix, t_final * 1e9)))
    )
    print('before:\n', params_before)
    print('after:\n', params_after)
    print('fidelity:\n', final_fidelity)

    # time evolution and signal after optimisation
    signal = generator.generate_signals(gate)[getDrive(model, q1).name]
    plotSignal(signal['ts'].numpy(), signal['values'].numpy(), directory + f"/signal_after_{file_suffix}.png",
               spectrum_cut=1e-3)
    populations = runTimeEvolutionDefault(exp, init_state, sequence)
    plotOccupations(exp, populations, sequence, filename=directory + f"/populations_after_{file_suffix}.png")

In [10]:
t_final = 30e-9
sigma = t_final / 20
sigma2 = sigma
relative_amp = np.sqrt(2)
freq = 5e9
anharm = 330e6
carrier_freq = 4.7e9

# TODO: target as index might not always work
optimisable_params = {
    "gauss": ["amp", "sigma",
              #"xy_angle", "freq_offset", # for single gaussian envelope
              #"t_final",
              "sigma2", "relative_amp",  # for double-gaussian envelope
              #"delta", # for DRAG
              ],
    "carrier": ["freq", "framechange"]
}

for num_gates in [1]:
    envelope = createDoubleGaussianPulse(t_final, sigma, sigma2, relative_amp)
    runExperiment(t_final, freq, anharm, carrier_freq,
                  envelope=envelope,
                  optimisable_params=optimisable_params,
                  algorithm=algorithms.cmaes,
                  algorithm_options={},
                  file_suffix=f"gaussian_{num_gates}gates",
                  directory="./output",
                  num_gates=num_gates
                  )

energies:  [5000000000.0, 4670000000.000003, 4339999999.999994, 4010000000.0000033]
final state:  [[ 0.32277481+0.31538154j]
 [ 0.15351267+0.39859758j]
 [ 0.68024393-0.38456644j]
 [-0.01405926+0.05548554j]
 [ 0.00126106-0.00191862j]]
final population:  tf.Tensor(
[[2.03649093e-01]
 [1.82446169e-01]
 [6.10623158e-01]
 [3.27630839e-03]
 [5.27139134e-06]], shape=(5, 1), dtype=float64)
saving plot in ./output/populations_before_gaussian_1gates.png
time domain:  (2999,) (2999,)
frequency domain:  (1500,) (1500,)
peaks:  [3.03383897e+09 4.60076679e+09 4.93415569e+09 6.93448908e+09]
saving plot in ./output/signal_before_gaussian_1gates.png
C3:STATUS:Saving as: /home/user/c3/output/log_gaussian_1gates_30.00/c1_state_transfer_infid_set_cmaes/2021_08_29_T_14_42_09/open_loop.log
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=349704, Sun Aug 29 14:42:09 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 4.717970889590769e-01 1.0e+00 9.46e-02  9e-02 